In [52]:
import cv2
import numpy as np
import pandas as pd
import copy
import json

In [2]:
def showImg(img, time=0):
    '''img = single np.array or list of np.array imgs
    displays all in window with window name = window(i) where i = index'''
    if type(img) != list:
        cv2.imshow("window", img)        
    else:
        for i, im in enumerate(img):
            cv2.imshow("window"+str(i), im)
    cv2.waitKey(time)
    if time==0:
        cv2.destroyAllWindows()

def threshImg(img, minThresh=100, maxThresh=255):   
    '''apply grayscale conversion and thresholding to image
    returns image''' 
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, imgThresh = cv2.threshold(imgGray, minThresh, maxThresh, cv2.THRESH_BINARY_INV)
    
    return imgThresh

# define template

## function to apply perspective transform and align forms 

In [3]:
def alignForm(img, width=0, height=0, template = False):
    '''apply perspective transformation to align form with template
    img = input form as np.array (use cv2.imread)
    width = use template image width (i.e. templateImg.shape[1])
    height = use template image height (i.e. templateImg.shape[0])
    template = if defining template: set to True to ensure the image is scaled down to 
    fit screen when showing image, will override width and height with euclidean distances of outer border'''
    

    # enhance image to improve contour detection
    
    #only resize if template - otherwise image will be resized with warp transform anyways
    xScale = 0.5
    yScale = 0.5
    if template == True:
        imgResize = cv2.resize(img, (0,0), fx= xScale, fy=yScale, interpolation= cv2.INTER_AREA)
        print("AlignImage: img resized by {}{} in width and {}{} in height".format(int(xScale*100),'%', int(yScale*100), '%'))
    else:
        imgResize = img
        
    imgGray = cv2.cvtColor(imgResize, cv2.COLOR_BGR2GRAY)
    imgBilat = cv2.bilateralFilter(imgGray, 11,500,0)
    imgEdges = cv2.Canny(imgBilat, 20,100 )
    
    ##find outer rectangle##
    
    conts = cv2.findContours(imgEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    #find top 10 biggest contours by area
    areas =[]
    for c in conts[0]:
        areas.append([c, cv2.contourArea(c)])


    sortedLengths = sorted(areas, key=lambda x:x[1], reverse=True)
    topConts = sortedLengths[:10]

    #find outer rectangle 
    outerBoxCnt = None
    for c in topConts:
        # approximate the contour
        peri = cv2.arcLength(c[0], True)
        #approximate curve to check if its rectangular
        approx = cv2.approxPolyDP(c[0], 0.015 * peri, True)
        if len(approx) == 4:
            outerBoxCnt = approx
            break

    pts = outerBoxCnt.reshape(4,2)

    #ordered from 0-3: top left, top right, bottom right, bottom left (go clockwise around rect)
    orderedPts = np.zeros((4,2), dtype='float32')

    #largest sum of x+y = bottom right
    #smallest sum of x+y = top left
    orderedPts[0] = pts[np.argmin(pts.sum(axis=1))]
    orderedPts[2] = pts[np.argmax(pts.sum(axis=1))]

    #smallest difference x-y = top right
    orderedPts[1] = pts[np.argmin(np.diff(pts, axis=1))]
    #largest difference x-y = bottom left
    orderedPts[3] = pts[np.argmax(np.diff(pts, axis=1))]

    #unpack ordered pts to find widths and heights
    (tl, tr, br, bl) = orderedPts

    #use euclidean distances (finally a use for pythagoras lol) - taken from pyimagesearch.com
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

    #find max heights/widths - i.e from top/bottom, left/right
    if width == 0 or template==True:
        maxWidth = max(int(widthA), int(widthB))
    else:
        maxWidth = width
    if height == 0 or template==True:
        maxHeight = max(int(heightA), int(heightB))
    else:
        maxHeight = height

    #initialise dst array for transformation
    dst = np.array([
        [0, 0],
        [maxWidth-1, 0],
        [maxWidth -1, maxHeight-1 ],
        [0, maxHeight-1]], dtype = "float32")

    #transformation matrix
    matrix = cv2.getPerspectiveTransform(orderedPts, dst)

    #transform image and resize to original size (map spots to correct locations)
    return cv2.warpPerspective(imgResize, matrix, (maxWidth, maxHeight))

# e.g. align template

In [4]:
templateImgPath = "scannedExceLPrint0.jpg"
tempImg =cv2.rotate(cv2.imread(templateImgPath),  cv2.ROTATE_90_CLOCKWISE)

tempImgAligned = alignForm(tempImg, template = True)

showImg(tempImgAligned)

AlignImage: img resized by 50% in width and 50% in height


# e.g. align form

In [5]:
oeeImgPath = "scannedExceLPrintForm0.jpg"
oeeImg = cv2.rotate(cv2.imread(oeeImgPath),  cv2.ROTATE_90_CLOCKWISE)

alignedOeeImg = alignForm(oeeImg, height= tempImgAligned.shape[0], width= tempImgAligned.shape[1], template=False)

showImg(alignedOeeImg)

## find x,y locs of spots in template

In [6]:
imgThresh = threshImg(tempImgAligned)
tempImgConts = tempImgAligned.copy()

conts, hier = cv2.findContours(imgThresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

spotCentres=[]
widths =[]
uniqueX =[]
uniqueY=[]
i=0
for c in conts: 
    i+=1
    # compute enclosing rect to get center points
    x,y,w,h = cv2.boundingRect(c)
    ar = w/h
    if 5<w<20 and 5<h<20 and 0.7<ar<1.3:      
        #draw on blank to show correct detection        
        cv2.rectangle(tempImgConts,(x,y), (x+w, y+h), (0,0,255))        
        widths.append(w)
        xCentre = int(x+(w/2))
        yCentre = int(y+(h/2))
        spotCentres.append([xCentre,yCentre])
#         if xCentre not in uniqueX:
#             uniqueX.append(xCentre)
#             cv2.putText(tempImgConts, "x=" + str(xCentre), (xCentre+5, yCentre+20), cv2.FONT_HERSHEY_PLAIN, 0.4, (0,0,255))
#         if yCentre not in uniqueY:
#             uniqueY.append(yCentre)
#             cv2.putText(tempImgConts, "y=" + str(yCentre), (xCentre+5, yCentre), cv2.FONT_HERSHEY_PLAIN, 0.4, (0,0,255))

        #cv2.putText(tempImgConts, str(xCentre)+","+str(yCentre), (xCentre+10, yCentre+10), cv2.FONT_HERSHEY_PLAIN, 0.5, (0,0,0))

#sort by x, then y
sortedSpots = sorted(spotCentres, key=lambda x:(x[0], x[1]), reverse=False)

print("spots found:", len(sortedSpots))


showImg(tempImgConts)
# cv2.imwrite("imgSpotCoordsY.jpg", tempImgConts)
# cv2.imwrite("imgSpotCoordsX.jpg", tempImgConts)

spots found: 707


## visualise x and y axis

In [7]:
imgCoord = alignForm(oeeImg, height= tempImgAligned.shape[0], width= tempImgAligned.shape[1], template=False)

xCoords = [xy[0] for xy in sortedSpots]
yCoords = [xy[1] for xy in sortedSpots]

maxX = max(xCoords)
maxY = max(yCoords)
minX = min(xCoords)
minY = min(yCoords)

xTest = list(np.linspace(minX, maxX, 30, dtype='int'))
yTest = list(np.linspace(minY, maxY, 20,dtype='int'))

for pt in tuple(zip(xTest, [100]*100)):
    x,y = pt
    cv2.circle(imgCoord, pt, 3, (0,0,255))
    cv2.putText(imgCoord, str(pt[0]), (x+5, y), cv2.FONT_HERSHEY_DUPLEX, 0.2, (0,0,255))
    cv2.line(imgCoord, (pt[0],0), (pt[0], 1000), (0,0,0), lineType=4)
    
for pt in tuple(zip([100]*100, yTest)):   
    x,y = pt
    cv2.circle(imgCoord, pt, 2, (0,0,255))
    cv2.putText(imgCoord, str(pt[1]), (x, y-5), cv2.FONT_HERSHEY_DUPLEX, 0.2, (0,0,255))
    cv2.line(imgCoord, (0,pt[1]), (1000, pt[1]), (0,0,0))
    

showImg(imgCoord)

## initialise list of dictionaries with: questions: list of responses dictionary
    ## i.e. list of (questions:answers dictionaries)
                answers = list of (answers:value, coord, filled) dictionaries

In [8]:
## access answers of question (use this to check filled status from image?)

def getQuestionDetails(questionString, templateList):
    '''return list of answer dictionaries for a question'''
    index = [ind for ind, qn in enumerate(templateList) if qn['question']==questionString]
    
    if len(index)==1:
        index = index[0]
    else:
        raise Exception("{} questions returned!".format(len(index)))
    return templateList[index]['answers']

# fill in template

### define generic function to generate questions for template

In [9]:
def generateQuestion(xRange, yRange, orient, questionName, answerValues, spotCoordsList):
    '''returns question dictionary 
    xRange/yRange = list of x and y coordinates
    orient = 'column' or 'row' or 'grid'
    questionName = str of question to associate with answerValues
    answerValues = list of values to associate with coordinates in xRange, 
    must be sorted from top to bottom (if orient = 'column'),
    left to right (if orient = 'row')
    left to right, then top to bottom - like reading english (if orient = 'grid')
    spotCoordsList = list of all coordinates from template to extract spots within xRange and yRange from.
    '''
    #return list of coordinates within range
    coordList = [i for i in spotCoordsList if xRange[0]<=i[0]<=xRange[1] and yRange[0]<=i[1]<=yRange[1]]
    
    #sort list by x if orient == 'row' or by y if column
    if orient =='row':
        coordList = sorted(coordList, key = lambda x:x[0])
    elif orient =='column':
        coordList = sorted(coordList, key = lambda x:x[1])
    elif orient=='grid':
        coordList = sorted(coordList,key =  lambda x:(x[1], x[0]))
    else:
        raise Exception("orient must be 'column', 'row', or 'grid'")
    
    
    #map coordinates to values
    if len(answerValues) == len(coordList):
        valueCoords = zip(answerValues, coordList)
    else:
        raise Exception("answerValues length: ({}) must match coordList length: ({})".format(len(answerValues), len(coordList)))
    
    #add question to template
    questionDict = {'question': questionName, 'answers':[]}
    
    #add in answers
    for val, coord in valueCoords:
        questionDict['answers'].append({'value': val, 'coord':coord, 'filled': False})
    
    return questionDict

## helper function to visualise questions/values

In [10]:
def showQuestion(questionString, templateList, img, fontSize, time=0, show=True):
    questionDetails = getQuestionDetails(questionString, templateList)
    questionLocX = questionDetails[0]['coord'][0]-10
    questionLocY = questionDetails[0]['coord'][1]-10
    #put text of question
    cv2.putText(img, questionString, (questionLocX, questionLocY), cv2.FONT_HERSHEY_SIMPLEX, fontSize*1.5, (0,0,0))
    for qn in questionDetails:
        x,y = qn['coord']        
        cv2.circle(img, (x,y), 5, (0,0,255), -1)        
        cv2.putText(img, str(qn['value']), (x+10, y), cv2.FONT_HERSHEY_SIMPLEX, fontSize, (0,0,0))
    if show:
        showImg(img, time)

### define questions
### need to "manually" define questions, values - use imgCoord to visualise x and y range

In [46]:
templateSpots=[]

In [47]:
showImg(imgCoord)

In [48]:
templateSpots.append(generateQuestion([35,40], [330,400] , 'column', 'day1', [0,1,2,3], sortedSpots))

In [49]:
templateSpots.append(generateQuestion([45,65], [330,520], 'column', 'day2', [0,1,2,3,4,5,6,7,8,9], sortedSpots))

In [50]:
templateSpots.append(generateQuestion([100, 120], [330, 550], 'column', 'month', list(range(1,13)), sortedSpots))

In [51]:
templateSpots.append(generateQuestion([150, 180], [320, 420], 'column', 'year', list(range(2020,2025)), sortedSpots))

In [52]:
templateSpots.append(generateQuestion([190, 220], [490, 530], 'column', 'shift', ['AM', 'PM'], sortedSpots))

In [53]:
templateSpots.append(generateQuestion([215,320], [490, 530], 'grid', 'cell', 
                 ['machined', 'isolast', 'custom', 'orings',
                  'bonded', 'flexVRings', 'silastics','cleanroom'], 
                 sortedSpots))

In [54]:
templateSpots.append(generateQuestion([290,600], [330, 400], 'grid', 'press', 
                 ['press1', 'press5', 'press9', 'press13', 
                  'press2', 'press6', 'press10', 'press14', 
                  'press3', 'press7', 'press11', 'press15', 
                  'press4', 'press8', 'press12', 'press16'], 
                 sortedSpots))

## loop through rows in 'lift counter' table

### moID

In [55]:
liftCounterYStart= 600
liftCounterYEnd =  765
liftCounterRows = np.linspace(liftCounterYStart, liftCounterYEnd, 10)

In [56]:
i=0 
for rw in liftCounterRows:
    i+=1
    templateSpots.append(generateQuestion([60,200],[rw-7, rw+7], 
                                          'row', 
                                          'hr{}MOID'.format(str(i)),
                                          list(range(1,7)),
                                          sortedSpots))


### lifts

In [57]:
i=0 
for rw in liftCounterRows:
    i+=1
    templateSpots.append(generateQuestion([200,430],[rw-7, rw+7], 
                                          'row', 
                                          'hr{}lifts'.format(str(i)),
                                          list(range(1,11)),
                                          sortedSpots))


### downtimecodes

In [58]:
i=0 
for rw in liftCounterRows:
    i+=1
    templateSpots.append(generateQuestion([440,600],[rw-7, rw+7], 
                                          'row', 
                                          'hr{}downtime'.format(str(i)),
                                          ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
                                          sortedSpots))


## define moid table spots

In [59]:
#[moid, xrange, yrange]
col1 = [675, 822]
col2 = [904, 1049]
row1= [132, 291]
row2 = [360, 518]
row3 = [587, 744]
moidTables= [[1,col1, row1],
            [2, col2, row1],
            [3, col1, row2],
            [4, col2, row2],
            [5, col1, row3],
            [6, col2, row3]]

In [60]:
for mo in moidTables:    
    moNum = mo[0]
    moidXRange = mo[1]
    moidYRange = mo[2]
    colNames = ['Pre', 'Suf1', 'Suf2', 'Suf3', 'Cav1', 'Cav2', 'Cav3']
    col1 = moidXRange[0]
    xCoords = np.linspace(mo[1][0], mo[1][1], 7)
    yCoords= np.linspace(mo[2][0], mo[2][1], 10)   

    i=0
    for x in xCoords:        
        if i == 0:
            templateSpots.append(generateQuestion([xCoords[i]-7,xCoords[i]+7], 
                                   moidYRange, 'column', 'moid{}{}'.format(moNum, colNames[i]), 
                     ['ORI', 'IAO', 'MCS', 'SIL', 'SEA', 'BON', 'BOM', 'CMO', 'SFALL', 'FAIR'], sortedSpots))
        
        else:
            templateSpots.append(generateQuestion([xCoords[i]-7,xCoords[i]+7], moidYRange, 'column', 'moid{}{}'.format(moNum,colNames[i]), 
                     list(range(0,10)), sortedSpots))
        i+=1
    

## animate through questions to show off question/answers

In [12]:
for qn in templateSpots:
    alignedOeeImg = alignForm(oeeImg, height= tempImgAligned.shape[0], width= tempImgAligned.shape[1], template=False)    
    showQuestion(qn['question'], templateSpots, alignedOeeImg, 0.3, time=1)

cv2.destroyAllWindows()

## export/import template as JSON

In [63]:
# with open("OEEv3.4Template.txt", 'w') as template:
#     json.dump(templateSpots, template)

In [31]:
with open("OEEv3.4Template.txt", 'r') as template:
    jsonTempLoad = json.load(template)
templateSpots = jsonTempLoad

## overlay template mask onto forms and find filled in spots

In [32]:
import os

In [33]:
imgList = [os.path.join("scanTrials", i) for i in os.listdir(r"scanTrials") if i.endswith(".jpg")]

In [49]:
finalAnswers=[]

for formPath in imgList:
    templateSpots =copy.deepcopy(jsonTempLoad)
    scannedFormImg =cv2.imread(formPath)
    if scannedFormImg.shape[0]>scannedFormImg.shape[1]:
        scannedFormImg=cv2.rotate(scannedFormImg, cv2.ROTATE_90_CLOCKWISE)
    alignedScanImg = alignForm(scannedFormImg, height= tempImgAligned.shape[0], width= tempImgAligned.shape[1], template=False)    
    alignedScanImgThresh = threshImg(alignedScanImg)

    #showImg(alignedScanImgThresh)

    filledThreshold =0.4
    circleSize = 6

    omrRead =[]

    for qn in templateSpots:
        #print('\n', qn['question'], '\n') 
        for ans in qn['answers']:
            #print(ans)


            mask =np.zeros(alignedScanImgThresh.shape, dtype = "uint8")
            cv2.circle(mask, tuple(ans['coord']), circleSize, 255, -1)        
            maskPixels = cv2.countNonZero(mask)
            mask = cv2.bitwise_and(alignedScanImgThresh, mask)
            pctFilled = cv2.countNonZero(mask)/maskPixels

            if pctFilled>=filledThreshold:
                ans['filled']=True
                cv2.circle(alignedScanImg, tuple(ans['coord']), circleSize, (0,0,255), 2)
                #print(qn['question'], ans['value'], pctFilled)
            else:
                cv2.circle(alignedScanImg, tuple(ans['coord']), circleSize, (0,100,0), 2)
            if pctFilled>0.1:
                cv2.putText(alignedScanImg, str(round(pctFilled, 2)), tuple(ans['coord']),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    showImg(alignedScanImg)

## return question:answer pairs

    filledSpots ={'form':formPath.split("\\")[-1]}

    for qn in templateSpots:    
        filledSpots[str(qn['question'])] = []
        for ans in qn['answers']:
            if ans['filled']==True:
                filledSpots[str(qn['question'])].append(ans['value'])

    finalAnswers.append(filledSpots)

## dict to dataframe (let the fun begin)

In [53]:
omrOutput = pd.DataFrame(finalAnswers)

In [113]:
def getDate(rw):
     return "".join([str(i) for i in rw['day1']]) + "".join([str(i) for i in rw['day2']])  + '-' +"".join([str(i) for i in rw['month']])+ '-'+ "".join([str(i) for i in rw['year']])

In [114]:
omrOutput['day']= omrOutput.apply(lambda x: getDate(x), axis=1)

In [115]:
omrOutput['day']

0     4-1-2024
1    3-16-2024
2           --
3     1-5-2024
4     3-3-2024
Name: day, dtype: object

## or mimic csv output style from formscanner to allow reuse of analysis?

## try align scanned image (fingers crossed!)

In [38]:
scantrialimg1 = cv2.rotate(cv2.imread('scanTrials10.jpg'), cv2.ROTATE_90_CLOCKWISE)
scantrialimg2 = cv2.rotate(cv2.imread('scanTrials11.jpg'), cv2.ROTATE_90_CLOCKWISE)

showImg(alignForm(scantrialimg2,height= tempImgAligned.shape[0], width= tempImgAligned.shape[1], template=False))

In [39]:
for qn in templateSpots:
    img = alignForm(scantrialimg1,height= tempImgAligned.shape[0], width= tempImgAligned.shape[1], template=False)
    showQuestion(qn['question'], templateSpots, img, 0.3, time=100)

cv2.destroyAllWindows()

## consolidate x and y vals

In [ ]:
consMargin =10

xCoords = [xy[0] for xy in sortedSpots]
yCoords = [xy[1] for xy in sortedSpots]

uniqueX = []
for x in xCoords:
    if x not in uniqueX:
        uniqueX.append(x)
        
uniqueY = []
for y in yCoords:
    if y not in uniqueY:
        uniqueY.append(y)

xIndices = np.where(np.diff(np.array(sorted(uniqueX)))<consMargin)
yIndices = np.where(np.diff(np.array(sorted(uniqueY)))<consMargin)

repeatedX = np.array(sorted(uniqueX))[list(xIndices[0])]

repeatedY = np.array(sorted(uniqueY))[list(yIndices[0])]

#define replacement lists, i.e. first index is value to replace, second index is replacement values 
##(order doesnt matter as long as its consistent)

replacementY=[]
for y in repeatedY:    
    replacementY.append([i for i in uniqueY if y-consMargin<i<y+consMargin])

replacementX=[]

for x in repeatedX:    
    replacementX.append([i for i in uniqueX if x-5<i<x+5])

replacementSpots = copy.deepcopy(sortedSpots)

for i in replacementSpots:
    #print("i", i)
    for vals in replacementX:        
        if i[0]==vals[0]:
            #print("match:", i[0], "to", vals[0], "changing to", vals[1])
            i[0]=vals[1]

for i in replacementSpots:
    #print("i", i)
    for vals in replacementY:        
        if i[1]==vals[0]:
            #print("match:", i[1], "to", vals[0], "changing to", vals[1])
            i[1]=vals[1]

#check if coordinates have been consolidated
xCoords = [xy[0] for xy in replacementSpots]
yCoords = [xy[1] for xy in replacementSpots]

replacementUniqueX = []
for x in xCoords:
    if x not in replacementUniqueX:
        replacementUniqueX.append(x)
        
replacementUniqueY = []
for y in yCoords:
    if y not in replacementUniqueY:
        replacementUniqueY.append(y)

#which x values have been removed?
print("x vals consolidated:", set(uniqueX)- set(replacementUniqueX))

#which y values have been removed?
print("y vals consolidated:",set(uniqueY)- set(replacementUniqueY))

## repeat consolidation (incase multiple spots within margin)

In [ ]:
consMargin =10

xCoords = [xy[0] for xy in replacementSpots]
yCoords = [xy[1] for xy in replacementSpots]

uniqueX = []
for x in xCoords:
    if x not in uniqueX:
        uniqueX.append(x)
        
uniqueY = []
for y in yCoords:
    if y not in uniqueY:
        uniqueY.append(y)

xIndices = np.where(np.diff(np.array(sorted(uniqueX)))<consMargin)
yIndices = np.where(np.diff(np.array(sorted(uniqueY)))<consMargin)

repeatedX = np.array(sorted(uniqueX))[list(xIndices[0])]

repeatedY = np.array(sorted(uniqueY))[list(yIndices[0])]

#define replacement lists, i.e. first index is value to replace, second index is replacement values 
##(order doesnt matter as long as its consistent)

replacementY=[]
for y in repeatedY:    
    replacementY.append([i for i in uniqueY if y-consMargin<i<y+consMargin])

replacementX=[]

for x in repeatedX:    
    replacementX.append([i for i in uniqueX if x-5<i<x+5])

replacementSpots = copy.deepcopy(replacementSpots)

for i in replacementSpots:
    #print("i", i)
    for vals in replacementX:        
        if i[0]==vals[0]:
            #print("match:", i[0], "to", vals[0], "changing to", vals[1])
            i[0]=vals[1]

for i in replacementSpots:
    #print("i", i)
    for vals in replacementY:        
        if i[1]==vals[0]:
            #print("match:", i[1], "to", vals[0], "changing to", vals[1])
            i[1]=vals[1]

#check if coordinates have been consolidated
xCoords = [xy[0] for xy in replacementSpots]
yCoords = [xy[1] for xy in replacementSpots]

replacementUniqueX = []
for x in xCoords:
    if x not in replacementUniqueX:
        replacementUniqueX.append(x)
        
replacementUniqueY = []
for y in yCoords:
    if y not in replacementUniqueY:
        replacementUniqueY.append(y)

#which x values have been removed?
print("x vals consolidated:", set(uniqueX)- set(replacementUniqueX))

#which y values have been removed?
print("y vals consolidated:",set(uniqueY)- set(replacementUniqueY))

In [324]:
oeeImg = cv2.warpPerspective(imgResize, matrix, (maxWidth, maxHeight))

xTestYLine = [30] * 100
yTestXLine = [10] * 100

xTest = list(zip(list(np.array(sorted(replacementUniqueX), dtype='int64')), xTestYLine))

yTest = list(zip(yTestXLine, list(np.array(sorted(replacementUniqueY), dtype='int64'))))


for pt in xTest:    
    cv2.circle(oeeImg, pt, 2, (0,0,255))
    cv2.putText(oeeImg, str(pt[0]), pt, cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0))
    cv2.line(oeeImg, (pt[0],0), (pt[0], 1000), (100,100,100), lineType=4)
    
for pt in yTest:    
    cv2.circle(oeeImg, pt, 2, (0,0,255))
    cv2.putText(oeeImg, str(pt[1]), pt, cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0))
    cv2.line(oeeImg, (0,pt[1]), (1000, pt[1]), (100,100,100))
    
#cv2.imwrite("imgCoord.jpg", oeeImg)
showImg(oeeImg)